In [60]:
# ============================================================
# Google Colab environment setup (pinned versions)
# ============================================================

import sys
import os
import subprocess

if "google.colab" in sys.modules:
    print("Running in Google Colab")
    print("Python version:", sys.version.split()[0])

    # ---- Required package versions --------------------------
    requirements = {
        "numpy": "2.4.0",
        "scipy": "1.16.3",
        "matplotlib": "3.10.8",
        "pandas": "2.3.3",
    }

    # ---- Check currently loaded versions --------------------
    restart_needed = False

    for pkg, required_version in requirements.items():
        try:
            module = __import__(pkg)
            installed_version = module.__version__
        except Exception:
            installed_version = None

        print(f"{pkg}: {installed_version} (required: {required_version})")

        if installed_version != required_version:
            restart_needed = True

    # ---- Install if needed ----------------------------------
    if restart_needed:
        print("\nInstalling pinned package versions...")

        pip_args = [
            f"{pkg}=={ver}" for pkg, ver in requirements.items()
        ]

        subprocess.check_call(
            [sys.executable, "-m", "pip", "install", "-q", *pip_args]
        )

        print("Installation complete.")
        print("Restarting runtime to load correct packages...")

        # This will appear as a "crash" in Colab — expected behavior
        os.kill(os.getpid(), 9)

    else:
        print("\nAll required package versions already installed.")

else:
    print("Not running in Google Colab — setup skipped.")
    print("Python version:", sys.version.split()[0])

Not running in Google Colab — setup skipped.
Python version: 3.12.12


In [61]:
# --- Version check ---
import numpy
import scipy
import matplotlib
import pandas

print("numpy:", numpy.__version__)
print("scipy:", scipy.__version__)
print("matplotlib:", matplotlib.__version__)
print("pandas:", pandas.__version__)

numpy: 2.4.0
scipy: 1.16.3
matplotlib: 3.10.8
pandas: 2.3.3


In [62]:
import numpy as np

def frame_element_kl(E, A, I, L):
    """
    Return 6x6 *local* stiffness matrix for a 2D frame element
    """

    factor = (E * I) / (L**3)

    kl = factor * np.array(
        [
            [ (A*L**2)/I,  0.0,      0.0,     -(A*L**2)/I,  0.0,      0.0],
            [ 0.0,         12.0,     6.0*L,    0.0,        -12.0,     6.0*L],
            [ 0.0,         6.0*L,    4.0*L**2, 0.0,        -6.0*L,    2.0*L**2],
            [-(A*L**2)/I,  0.0,      0.0,      (A*L**2)/I,  0.0,      0.0],
            [ 0.0,        -12.0,    -6.0*L,    0.0,         12.0,    -6.0*L],
            [ 0.0,         6.0*L,    2.0*L**2, 0.0,        -6.0*L,    4.0*L**2],
        ],
        dtype=float,
    )

    return kl

In [63]:
def resolve_load_to_local(f_global, theta):
    """
    Resolve a global vertical load into the element's local coordinate system.

    The load is resolved using the element's global orientation angle (theta).

    Sign conventions:
        - Perpendicular loads: "down" on the beam (opposite the local +y axis)
          are defined as positive.
        - Parallel loads: forces opposite the local +x axis
          are defined as positive.

    Resolution equations:
        f_x =  f_global * sin(theta)
        f_y =  f_global * cos(theta)

    Parameters
    ----------
    f_global : float
        Global load magnitude (kN or kN/m).
    theta : float
        Element orientation angle in degrees (measured from global X).

    Returns
    -------
    (f_x, f_y) : tuple of floats
        Load components in the local coordinate system.
    """
    theta = np.radians(theta)  # Convert degrees → radians

    f_x = f_global * np.sin(theta)
    f_y = f_global * np.cos(theta)

    return f_x, f_y

In [64]:
def fef_perpendicular(load_type, L, *, wy=None, Py=None, a=None):
    """
    Fixed-end forces (local) for a fixed-fixed beam/frame member under transverse loading
    in the local y-direction.

    DOF / force order:
        [F1, F2, F3, F4, F5, F6] = [N_i, V_i, M_i, N_j, V_j, M_j]
    (Axial terms N_i, N_j are zero here.)

    Sign convention:
      - wy and Py are SIGNED in local +y.
      - If your load is downward while +y is up, pass wy < 0 or Py < 0.
      - End moments come out with opposite signs at i and j.

    Parameters
    ----------
    load_type : str
        "uniform" or "point"
    L : float
        Member length
    wy : float, optional
        Uniform load intensity (kN/m) in local y (signed). Required if load_type="uniform".
    Py : float, optional
        Point load magnitude (kN) in local y (signed). Required if load_type="point".
    a : float, optional
        Distance from i-end to the point load (same units as L). Required if load_type="point".

    Returns
    -------
    np.ndarray shape (6,)
        Local fixed-end force vector [N_i, V_i, M_i, N_j, V_j, M_j]
    """
    load_type = load_type.lower().strip()

    # Axial components (none for pure transverse loading here)
    F1 = 0.0
    F4 = 0.0

    if load_type in ("uniform", "distributed", "udl"):
        if wy is None:
            raise ValueError("For load_type='uniform', you must provide wy.")

        F2 = wy * L / 2.0
        F3 = wy * L**2 / 12.0
        F5 = wy * L / 2.0
        F6 = -wy * L**2 / 12.0

        return np.array([F1, F2, F3, F4, F5, F6], dtype=float)

    elif load_type in ("point", "pl", "concentrated"):
        if Py is None or a is None:
            raise ValueError("For load_type='point', you must provide Py and a.")
        if not (0.0 <= a <= L):
            raise ValueError(f"a must satisfy 0 <= a <= L. Got a={a}, L={L}.")

        b = L - a

        F2 = Py * b**2 * (3.0*a + b) / L**3
        F3 = Py * a * b**2 / L**2
        F5 = Py * a**2 * (a + 3.0*b) / L**3
        F6 = -Py * a**2 * b / L**2

        return np.array([F1, F2, F3, F4, F5, F6], dtype=float)

    else:
        raise ValueError("load_type must be one of: 'uniform' or 'point'.")

In [65]:
def fef_parallel(load_type, L, *, wx=None, Px=None, a=None):
    """
    Fixed-end forces (local) for a fixed-fixed member under loading PARALLEL
    to the local x-axis (axial loading).

    DOF / force order:
        [F1, F2, F3, F4, F5, F6] = [N_i, V_i, M_i, N_j, V_j, M_j]

    Sign convention:
      - PARALLEL loads: forces opposite the local +x axis (compressive) are POSITIVE.
      - So if your physical load acts in +x tension, pass wx < 0 or Px < 0.

    Parameters
    ----------
    load_type : str
        "uniform" or "point"
    L : float
        Member length
    wx : float, optional
        Uniform axial load intensity (kN/m) along local x (signed). Required if "uniform".
    Px : float, optional
        Point axial load (kN) along local x (signed). Required if "point".
    a : float, optional
        Distance from i-end to point load (same units as L). Required if "point".

    Returns
    -------
    np.ndarray shape (6,)
        Local fixed-end force vector [N_i, V_i, M_i, N_j, V_j, M_j]
    """
    load_type = load_type.lower().strip()

    # No shear/moment from pure axial loading
    F2 = 0.0
    F3 = 0.0
    F5 = 0.0
    F6 = 0.0

    if load_type in ("uniform", "distributed", "udl"):
        if wx is None:
            raise ValueError("For load_type='uniform', you must provide wx.")

        F1 = wx * L / 2.0
        F4 = wx * L / 2.0

        return np.array([F1, F2, F3, F4, F5, F6], dtype=float)

    elif load_type in ("point", "pl", "concentrated"):
        if Px is None or a is None:
            raise ValueError("For load_type='point', you must provide Px and a.")
        if not (0.0 <= a <= L):
            raise ValueError(f"a must satisfy 0 <= a <= L. Got a={a}, L={L}.")

        # Consistent nodal forces from axial bar shape functions
        F1 = Px * (L - a) / L
        F4 = Px * a / L

        return np.array([F1, F2, F3, F4, F5, F6], dtype=float)

    else:
        raise ValueError("load_type must be one of: 'uniform' or 'point'.")

---
---

In [66]:
# --- properties in kN and m ---
E = 100e6          # kN/m^2  (converted from 200e9 N/m^2)
A = 15000e-6       # m^2 (converted from 12500 mm^2)
I = 300e-6         # m^4 (converted from 275e6 mm^4)
L = 13.0            # m

# displacement vector (units: m, rad)
u = np.array([
    0.015,
    -0.02,
    -0.001, # Clockwise rotation is negative
    0.02,
    0.06,
    0.005 # Clockwise rotation is negative
], dtype=float)

np.set_printoptions(precision=2, suppress=True)

In [67]:
k_local = frame_element_kl(E, A, I, L)

Q = k_local @ u
print(Q)

[-576.92   -8.85  -71.36  576.92    8.85  -43.67]


In [68]:
theta = 90 + np.degrees(np.arctan(12/5))  # degrees

f = 100 # kN

f_x, f_y = resolve_load_to_local(f, theta)


print(f"f_x (kN): {f_x:.1f}")
print(f"f_y (kN): {f_y:.2f}")

f_x (kN): 38.5
f_y (kN): -92.31


In [69]:
# # Uniform distributed load perpendicular to the member
# Qf_y = fef_perpendicular("uniform", L, wy=f_y)
# print(Qf_y)

# Point load downward at a = 2 m from i
Qf_y  = fef_perpendicular("point", L, Py=f_y, a=0.7*L)
print(Qf_y)

[  0.   -19.94 -75.6    0.   -72.37 176.4 ]


In [70]:
# # Uniform distributed load parallel to the member
# Qf_x = fef_parallel("uniform", L, wx=f_x)
# print(Qf_x)

# Point load parallel to the member at a = distance from i
Qf_x  = fef_parallel("point", L, Px=f_x, a=0.7*L)
print(Qf_x)

[11.54  0.    0.   26.92  0.    0.  ]


In [71]:
# --- total FEF vector ---
Qf = Qf_x + Qf_y
print(Qf)

[ 11.54 -19.94 -75.6   26.92 -72.37 176.4 ]


In [72]:
# --- Local Element Forces ---
Q_total = Q+ Qf
print(Q_total)

[-565.38  -28.79 -146.96  603.85  -63.52  132.73]
